# Algoritmo de desarrollo orientado a objetos y numpy
Se busca probar las funciones y scripts para obtener una mayor rapidez de ejecucion.
Se generan los algoritmos necesarios para el desarrollo del algoritmo que compare ambas estructuras
Basandose en la estructura como esta en start_oo.py

In [24]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist, squareform
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime
#distancia
from scipy.spatial import distance
#multiprocesamiento
import multiprocessing
from functools import partial

In [25]:
# assert( len(sys.argv) > 1)
# lectura de archivo
file1 = 'pdbs/1xxa.pdb' # sys.argv[1]
file2 = 'pdbs/1tig.pdb' # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo...
num_cliques = 3

# se define la estructura
pdb1 = rpt.PdbStruct("first")
pdb2 = rpt.PdbStruct("second")

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()
# se pega a la tabla la estructura secundaria
ss1 = pdb1.Get_SS(file1)
ss2 = pdb1.Get_SS(file2)

In [26]:
# se crea atributo a cada residuo de la estructura secundaria
for i, j in zip(pdb11, ss1.structure.values):
    setattr(i, 'structure', j)
for i, j in zip(pdb22, ss2.structure.values):
    setattr(i, 'structure', j)

In [27]:
def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada proteina"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    index = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = []
    # se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v in coord:
        distancia_un_atomo = []
        for av in coord:
            distancia = pdist(np.array([v, av]), metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=index, columns=index, data=distancias)
    return(df_da, index)


df_distancias1, index1 = get_df_distancias(pdb11)
df_distancias2, index2 = get_df_distancias(pdb22)

In [28]:
# se generan cliques, tte devuleve dataframe con cliques de 3 y la lista de cliques sin partir
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1, dth=10, k=num_cliques)
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2, dth=10, k=num_cliques)
print('**'*50)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [29]:
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

In [30]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
df_cliques1 = fc.paste_SS(ss1, df_cliques1, num_cliques = num_cliques)
df_cliques2 = fc.paste_SS(ss2, df_cliques2, num_cliques = num_cliques)

In [31]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(df_cliques1,df_cliques2, num_cliques=num_cliques)

In [32]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, df_cliques2, num_cliques)

In [33]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [34]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [35]:
for i,j in enumerate(df_cliques1.columns):
    print(i,j)

0 0
1 1
2 2
3 ss_0
4 ss_1
5 ss_2
6 coord_clique_0
7 coord_clique_1
8 coord_clique_2
9 matriz_coordenadas
10 baricentro_clique
11 vec_gorro_0
12 vec_gorro_1
13 vec_gorro_2
14 vectores_gorro


In [36]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [37]:
#calculo del RMSD
print(len(candidatos_ss))

1705808


# Codigo que restringe candidatos

In [38]:
a = (0,0,0)
df_cliques1['distancia_promedio'] = [np.mean([distance.euclidean(a, i[0]),distance.euclidean(a, i[1]),distance.euclidean(a, i[2])]) for i in df_cliques1.vectores_gorro]
df_cliques2['distancia_promedio'] = [np.mean([distance.euclidean(a, i[0]),distance.euclidean(a, i[1]),distance.euclidean(a, i[2])]) for i in df_cliques2.vectores_gorro]

array_dist_promedio1 = df_cliques1.values[:, 15] #del 9 al 15
array_dist_promedio2 = df_cliques2.values[:, 15]

In [39]:
candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -0.5) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= 0.5)]

print(len(candidatos_ss))
print(len(candidatos_filter_dist))

1705808
280640


In [40]:
%%time
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.81 µs


# Haciendo MUltiprocessing

In [41]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)

In [42]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 7.42 s, sys: 72.6 ms, total: 7.49 s
Wall time: 15.5 s


In [43]:
len(rmsd_1)

280640

In [60]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(280640, 5)

In [61]:
f1[f1.cumple==1].cumple.sum()

6907

# Probando la generacion de una nueva tabla con cliques_4 si se empieza con cliques_3

In [71]:
candidatos_df = f1
candidatos_df.sort_values(['candidato_clique_1','candidato_clique_2']).head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
0,1.70200,"(1, 1060)",0,1,1060
1,1.43155,"(3, 820)",0,3,820
2,1.05198,"(3, 824)",0,3,824
3,1.34365,"(3, 1027)",0,3,1027
4,0.97235,"(3, 1033)",0,3,1033


In [55]:
df_cliques1[[0,1,2]].head()

,0,1,2
0,96,97,132
1,89,97,132
2,97,98,132
3,97,91,132
4,96,89,132


In [56]:
cliques1[:5]

[[132, 97, 96, 89, 98, 91],
 [132, 97, 96, 89, 98, 99, 131],
 [132, 97, 96, 133, 95, 98, 91],
 [132, 97, 96, 133, 95, 98, 131],
 [132, 97, 96, 133, 95, 131, 124]]

In [96]:
#fila,columna
set_clique_1 = [df_cliques1.iloc[i, [0,1,2]].values for i in candidatos_df.candidato_clique_1[:10]]

In [139]:
lista_chidi = []
muestra = set_clique_1[0]
for i in cliques1[:100]:
    print(set(i))
    print(set(muestra))
    
    if set(muestra).issubset(i):       
        print('estoy')
        no_estan_en_clique = set(i).difference(set(muestra))
        print(no_estan_en_clique)
        print(set(muestra).update(no_estan_en_clique))
        print('--'*10)
    print('--'*10)

{96, 97, 98, 132, 89, 91}
{89, 132, 97}
estoy
{96, 98, 91}
None
--------------------
--------------------
{96, 97, 98, 99, 132, 131, 89}
{89, 132, 97}
estoy
{96, 98, 99, 131}
None
--------------------
--------------------
{96, 97, 98, 132, 133, 91, 95}
{89, 132, 97}
--------------------
{96, 97, 98, 131, 132, 133, 95}
{89, 132, 97}
--------------------
{96, 97, 131, 132, 133, 124, 95}
{89, 132, 97}
--------------------
{96, 97, 132, 133, 134, 91, 94, 95}
{89, 132, 97}
--------------------
{96, 97, 132, 133, 134, 124, 95}
{89, 132, 97}
--------------------
{97, 130, 99, 132, 98, 131, 89}
{89, 132, 97}
estoy
{130, 99, 131, 98}
None
--------------------
--------------------
{97, 130, 99, 132, 98, 131, 126}
{89, 132, 97}
--------------------
{97, 98, 131, 132, 133, 126}
{89, 132, 97}
--------------------
{97, 131, 132, 133, 124, 126}
{89, 132, 97}
--------------------
{98, 131, 132, 130, 109, 126}
{89, 132, 97}
--------------------
{98, 131, 132, 133, 109, 126}
{89, 132, 97}
--------------

In [125]:
a = set([1,2,3])
a.

{1, 2, 3}